In [1]:
import pandas as pd
import pickle as pkl
from xgboost import XGBRegressor
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [2]:
file = './Data/dataset.csv'

In [3]:
df = pd.read_csv(f'{file}')\
       .fillna(0)

# https://xgboost.readthedocs.io/en/stable/parameter.html
xgb = XGBRegressor(booster='gbtree',
                   n_estimators=25,
                   learning_rate=0.15, # default=0.3
                   max_depth=3, # default=6
                   
                   reg_lambda=4, # default=1 # L2 regularization
                   
                   tree_method='approx', # default='auto' aka 'exact'
                   grow_policy='lossguide', # approx only
                   max_leaves=5, #default=0, # approx only
                   max_bin=512, #default=256, # approx only
                   random_state=0)

def count_y(df):
    num_y = len(df.columns[3:])
    return num_y

def prepare_dataset(df, num_y, train_interval):
    for label, content in df.iloc[:,3:(3+num_y)].items(): # cols 0:2 are datetime cols # can 3+num_y be changed to just -1?    
        col_name = label+f" lag+{train_interval}"
        df[col_name] = df[label].shift(train_interval)
    df.drop(index=df.index[[0]], axis=0, inplace=True) # lag cols have null rows
    return df

def train_test_split(df, num_y, test_rows):
    X = pd.concat((df['Timestep'], df.iloc[:,(3+num_y):]), axis=1) # timestep, lag X1, lag X2
    y = df.iloc[:, 3:(3+num_y)] # actual X1, actual X2

    num_rows = len(df.index)
    split_rows = num_rows - test_rows
    
    X_train = X.iloc[:split_rows]
    X_test = X.iloc[split_rows:]
    y_train = y.iloc[:split_rows]
    y_test = y.iloc[split_rows:]
    
    return [X, X_train, X_test, y, y_train, y_test]

def get_results(model, X, X_train, X_test, y, y_train, y_test):
    # Run model
    model = model.fit(X_train,y_train)
    predictions = model.predict(X_test)
    df_valid = pd.concat([X_test.Timestep, y_test],axis=1)
    
    X_test.reset_index(drop=True, inplace=True)
    df_y_preds = pd.DataFrame(predictions, columns=y.columns)
    df_preds = pd.concat([X_test.Timestep, df_y_preds], axis=1)
    
    return model, df_preds, df_valid

num_y = count_y(df)
train_interval = 12 # months to shift for lag columns # 12 gives best performance
df = prepare_dataset(df, num_y, train_interval)
test_rows = 5 # rows in the dataset that're not for training
tts_list = train_test_split(df, num_y, test_rows)
model, df_preds, df_valid = get_results(xgb, *tts_list)

display("Actuals:",df_valid,
        "Predictions:",df_preds)

'Actuals:'

,Timestep,A1,B1,C1,D1,E1,F1,G1,H1,I1,...,A2,B2,C2,D2,E2,F2,G2,H2,I2,J2
77,78,2142.5,616.4,2146.6,498.9,416.2,111.5,20994.0,17207.0,7805.0,...,1777,135,3359,183,534,19.0,10730,2687,838,501
78,79,2120.3,520.0,2013.8,370.0,394.9,432.3,20465.0,19480.0,9316.0,...,1592,122,3110,132,466,51.0,10525,2633,969,474
79,80,2187.6,852.1,2160.2,400.5,504.7,200.4,29227.0,18565.0,8922.0,...,1851,152,3384,171,706,142.0,12963,2833,1429,469
80,81,2043.3,797.4,2528.1,526.1,484.7,463.1,16531.9,17664.1,6577.7,...,1783,170,3199,159,952,85.0,7150,2713,545,399
81,82,1822.3,1080.6,2192.8,413.7,736.8,299.0,14620.4,14705.8,5832.7,...,1580,197,3129,159,1143,152.0,6800,2635,467,398


'Predictions:'

,Timestep,A1,B1,C1,D1,E1,F1,G1,H1,I1,...,A2,B2,C2,D2,E2,F2,G2,H2,I2,J2
0,78,1704.774170,804.572510,2130.046631,390.480164,392.226868,100.267166,21770.644531,12341.163086,6251.287598,...,1711.361938,207.697327,2850.224365,139.724777,700.687378,17.893074,10088.442383,2405.543213,791.522156,352.084808
1,79,1808.200684,738.215332,2130.046631,365.784088,340.679199,109.836723,25146.480469,12036.032227,6071.300293,...,1554.525513,238.618225,2761.366943,135.115173,605.188232,30.010580,7115.916992,2238.628418,612.454407,345.645111
2,80,1436.302246,725.188660,2117.963867,387.438293,386.191864,100.936295,18697.460938,11697.129883,4935.188965,...,1314.789307,167.058701,2535.916016,143.572571,754.067444,17.291162,9683.092773,2389.021484,494.133972,251.128845
3,81,1530.706299,716.571045,2130.046631,390.130981,431.680939,95.107376,17967.669922,8631.333008,4780.489746,...,1321.965576,188.630051,2821.939209,145.365372,859.664246,17.291162,7489.082031,2473.566406,312.242523,352.319916
4,82,1588.990479,737.624268,2130.046631,381.330994,658.211792,87.053566,27199.398438,8389.782227,5621.973633,...,1211.456909,163.557022,2831.188721,139.737091,801.965515,15.463931,7772.947754,2146.727539,599.530151,347.738770


In [4]:
df_var = df_valid.div(df_preds.values)
display("Percent Variance:",df_var) # standard metric key performance indicator

'Percent Variance:'

,Timestep,A1,B1,C1,D1,E1,F1,G1,H1,I1,...,A2,B2,C2,D2,E2,F2,G2,H2,I2,J2
77,1.0,1.256765,0.766121,1.007771,1.277658,1.061121,1.112029,0.964326,1.394277,1.248543,...,1.038354,0.649984,1.178504,1.309718,0.762109,1.061863,1.063593,1.117003,1.058720,1.422953
78,1.0,1.172602,0.704402,0.945425,1.011526,1.159155,3.935842,0.813832,1.618474,1.534432,...,1.024107,0.511277,1.126254,0.976944,0.770008,1.699401,1.479079,1.176167,1.582159,1.371349
79,1.0,1.523078,1.175005,1.019942,1.033713,1.306863,1.985411,1.563153,1.587141,1.807834,...,1.407830,0.909860,1.334429,1.191035,0.936256,8.212288,1.338725,1.185841,2.891928,1.867567
80,1.0,1.334874,1.112800,1.186875,1.348521,1.122820,4.869233,0.920091,2.046509,1.375947,...,1.348749,0.901235,1.133618,1.093796,1.107409,4.915806,0.954723,1.096797,1.745438,1.132493
81,1.0,1.146829,1.464973,1.029461,1.084884,1.119397,3.434667,0.537527,1.752823,1.037483,...,1.304215,1.204473,1.105189,1.137851,1.425248,9.829325,0.874829,1.227450,0.778943,1.144537


In [5]:
df_preds.to_excel('./Outputs/e_em_xgb_lambda4_junoct22_preds_export.xlsx')
df_valid.to_excel('./Outputs/e_em_xgb_lambda4_junoct22_valid_export.xlsx')
df_var.to_excel('./Outputs/e_em_xgb_lambda4_junoct22_var_export.xlsx')

In [6]:
pkl.dump(model, open('./Model/model.pkl','wb'))